In [4]:

import pathlib

import numpy as np
from matplotlib.axes import Axes
!pip install matplotlib pandas numpy-financial
import matplotlib.pyplot as plt



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from matplotlib.ticker import FuncFormatter


class Mortgage:
    def __init__(
            self,
            principal,
            annual_interest_rate,
            annual_tr,
            max_months,
            monthly_payment,
            fgts_annual=0,
            fgts_frequency=24,
            min_monthly_valorization=0.001,
            max_monthly_valorization=0.008,
            adhoc=None
    ):
        if adhoc is None:
            adhoc = dict()
        self.principal = principal
        self.annual_interest_rate = annual_interest_rate
        self.annual_tr = annual_tr
        self.monthly_payment = monthly_payment
        self.fgts_annual = fgts_annual
        self.max_months = max_months
        self.fgts_frequency = fgts_frequency
        self.min_monthly_valorization = min_monthly_valorization
        self.max_monthly_valorization = max_monthly_valorization
        self.adhoc = adhoc

    def calculate(
            self,
    ):
        monthly_interest_rate = (self.annual_interest_rate + self.annual_tr) / 12 / 100

        payments = []
        balances = [self.principal]
        interests = []
        total_paid = 0
        amortizations = []
        current_balance = self.principal
        month = 0
        fgts_payments = []
        valorization = []
        min_value = current_balance
        max_value = current_balance
        while current_balance > 0:
            total_payment = self.monthly_payment

            if month % self.fgts_frequency == 0 and 0 < self.fgts_annual <= current_balance:
                total_payment += self.fgts_annual
                fgts_payments.append(month)
            adhoc_pay = self.adhoc.get(month, None)
            if adhoc_pay is not None:
                total_payment += adhoc_pay

            interest = current_balance * monthly_interest_rate
            amortization = total_payment - interest

            if amortization > current_balance:
                amortization = current_balance

            new_balance = current_balance - total_payment + interest
            min_value *= (1 + self.min_monthly_valorization)
            max_value *= (1 + self.max_monthly_valorization)

            amortizations.append(amortization)
            payments.append(total_payment)
            balances.append(new_balance)
            interests.append(interest)
            valorization.append((min_value, max_value))

            total_paid += total_payment
            current_balance = new_balance
            month += 1
            if month > self.max_months:
                break
        return payments, balances, interests, amortizations, fgts_payments, valorization, total_paid


configurations = [
        Mortgage(
        240000,
        9.99,
        0,
        360,
        2578.63
        ),
    Mortgage(
        321252.03,
        8.27,
        1.76,
        342,
        3270.64,
        adhoc={
            20: 50000,
            35: 35000,
            45: 10000,
            80: 50000
        }),
    Mortgage(
        321252.03,
        8.27,
        1.76,
        342,
        5000,
        adhoc={
            20: 50000,
            35: 35000,
            45: 10000,
            80: 50000
        }),
    Mortgage(
        321252.03,
        8.27,
        1.76,
        342,
        8000,
        adhoc={
            20: 50000,
            35: 35000,
            45: 10000,
            80: 50000
        })
]
for principals in [300000, 450000, 665000, 800000]:
    for interest_rate_over_year in [6.5, 8.4, 10.8]:
        for monthly_payment in [3000, 5000, 8000, 15000, 20000]:
            for years in [10, 30]:
                for fgts in [0, 14400]:
                    configurations.append(
                        Mortgage(principals, interest_rate_over_year, 1.76, years * 12, monthly_payment, fgts),
                    )


In [6]:

for mortgage in configurations:
    fig = plt.figure(figsize=(30, 15))
    ax1 = plt.gca()  # Get current axis for primary y-axis
    ax2: Axes = ax1.twinx()  # Create a secondary y-axis sharing the same x-axis
    min_valorization = 10
    max_valorization = 48
    payments, balances, interests, amortizations, fgts_payments, valorization, total_paid = mortgage.calculate()
    total_interest = np.sum(interests)
    total_num_payments = len(payments)
    ax2.plot(
        range(0, total_num_payments),
        np.cumsum(interests) + mortgage.principal,
        color='orange',
        label="Loan paid with interest over time"
    )
    ax1.stackplot(
        range(total_num_payments),
        [amortizations, interests],
        labels=['Amortization', 'Interest'],
        colors=['green', 'red'],
        alpha=0.5,
        step="post"
    )

    min_val = [minimum for (minimum, _) in valorization]
    max_val = [maximum for (_, maximum) in valorization]
    avg_val = [(minimum + maximum) / 2 for minimum, maximum in valorization]  # Calculate the average

    ax2.fill_between(
        range(total_num_payments),
        min_val,
        max_val,
        alpha=0.25,
        label="Possible valorization"
    )
    ax2.plot(
        range(total_num_payments),
        avg_val,
        'k-',  # 'k-' stands for black solid line
        label="Average possible valorization"
    )

    ax2.plot(fgts_payments, marker='o', color='yellow', label=f'FGTS (BRL {mortgage.fgts_annual})')
    ax2.plot(balances, label="Balance", color='red')
    ax1.set_title(
        f"{mortgage.principal} BRL, {mortgage.max_months} months, monthly payment of {mortgage.monthly_payment}, biannual FGTS of {mortgage.fgts_annual}, Annual interest: {mortgage.annual_interest_rate}: BRL {total_paid:,} ({total_paid / mortgage.principal:,}x), Total Interest: {total_interest}")
    ax1.set_xlabel('Months')
    ax1.set_ylabel('Amount in BRL (Payments, Interest, Amortization)')
    ax2.set_ylabel('Amount in BRL (Remaining Balance)')

    # Legend and grid setup
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines + lines2, labels + labels2, loc='upper right')
    ax1.grid(True)

    # Format y-axis to be more readable
    formatter = FuncFormatter(lambda y, _: f'{int(y):,}')
    ax1.yaxis.set_major_formatter(formatter)
    ax2.yaxis.set_major_formatter(formatter)

    path = pathlib.Path(
        f"charts/{mortgage.principal}/{mortgage.monthly_payment}BRL month/${mortgage.annual_interest_rate}/{mortgage.fgts_annual} biannual FGTS.png")
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(path)
    plt.close()